In [83]:
import json
from pprint import pprint
import pandas as pd
import requests
import datetime as dt

def get_liquidities(pool="0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640", cutoff_timestamp=1668227632, subgraph_url="https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3"):
    """
    Get all of the Aave liquidations, back
    to a specific time `cutoff_timestamp`.
    
    cutoff_timestamp: int
    """
    url = subgraph_url
    
    sup_timestamp = dt.datetime.now().timestamp() + 100 # greates that the timestamp can be, on each round of pagination
    all_liquidities = []
    
    for i in range(1_000):
        query = '''
        {
            poolHourDatas(
                first:200,
                orderBy:periodStartUnix,
                orderDirection:desc,
                where:{
                  pool:"%s",
                  periodStartUnix_lte:%d
                }
            ) {
                liquidity
                periodStartUnix
            }
        }
        ''' % (pool, sup_timestamp)

        result = json.loads(requests.post(
            url,
            json={"query": query}
        ).content)

        if ("data" not in result) or (len(result["data"]["poolHourDatas"]) == 0):
            pprint(result)
            print("breaking due to error in response data")
            return

        result = json.loads(requests.post(
            url,
            json={"query": query}
        ).content)

        if ("data" not in result) or (len(result["data"]["poolHourDatas"]) == 0):
            pprint(result)
            print("breaking a")
            break

        resdata = result["data"]["poolHourDatas"]


        # linear-time search for the last timestamp; can be optimized to O(log(n)), but not worth it rn
        j = len(resdata)
        while int(resdata[j-1]["periodStartUnix"]) < cutoff_timestamp:
            j -= 1

        if sup_timestamp == int(resdata[j-1]["periodStartUnix"]):
            # note, this could be problematic if we had >1,000 swaps that all had the same timestamp
            print("done, since we have the same sup_timestamp as before")
            break
        else:
            #print("unixes", [x["periodStartUnix"] for x in resdata])
            sup_timestamp = int(resdata[j-1]["periodStartUnix"]) # set the maximum time
            print("sup timestamp", dt.datetime.fromtimestamp(sup_timestamp), "\n")

        # concatenate all swaps that have timestamp >= cutoff_timestamp
        all_liquidities += resdata[:j]
        if j != len(resdata):
            print("done, since the final swap is in the interior of the result batch")
            break
            
    
    

    day_datas = pd.DataFrame(all_liquidities).set_index("periodStartUnix")

    # put the liquidities human-readable format, i.e. decimal scale them down
    day_datas.liquidity = day_datas.liquidity.apply(lambda x: int(x)/(10**12))
            
    return day_datas

In [84]:
liquidities_ = get_liquidities(
    cutoff_timestamp=dt.datetime(2022, 7, 1).timestamp()
)

sup timestamp 2022-12-28 16:00:00 

sup timestamp 2022-12-20 09:00:00 

sup timestamp 2022-12-12 02:00:00 

sup timestamp 2022-12-03 19:00:00 

sup timestamp 2022-11-25 12:00:00 

sup timestamp 2022-11-17 05:00:00 

sup timestamp 2022-11-08 22:00:00 

sup timestamp 2022-10-31 16:00:00 

sup timestamp 2022-10-23 09:00:00 

sup timestamp 2022-10-15 02:00:00 

sup timestamp 2022-10-06 19:00:00 

sup timestamp 2022-09-28 12:00:00 

sup timestamp 2022-09-20 05:00:00 

sup timestamp 2022-09-11 22:00:00 

sup timestamp 2022-09-03 15:00:00 

sup timestamp 2022-08-26 08:00:00 

sup timestamp 2022-08-18 01:00:00 

sup timestamp 2022-08-09 18:00:00 

sup timestamp 2022-08-01 11:00:00 

sup timestamp 2022-07-24 04:00:00 

sup timestamp 2022-07-15 21:00:00 

sup timestamp 2022-07-07 14:00:00 

sup timestamp 2022-07-01 00:00:00 

done, since the final swap is in the interior of the result batch


In [85]:
liquidities_

,liquidity
periodStartUnix,
1672977600,5.117340e+07
1672974000,5.117340e+07
1672970400,5.058007e+07
1672966800,6.194900e+07
1672963200,4.292250e+07
...,...
1656662400,1.412491e+07
1656658800,1.346404e+07
1656655200,1.791447e+07


In [86]:
liquidities_.to_csv(
    "./data/hourly-liquidities.csv"
)

In [78]:
dt.datetime.fromtimestamp(1669870800)

datetime.datetime(2022, 12, 1, 0, 0)